# Single-cell kymograph maker
This notebook is for making kymographs focused on a single apoptotic scribble cell scanning over just wild-type cells

In [23]:
import btrack
print("version no.:", btrack.__version__)
from btrack.utils import import_HDF, import_JSON
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import random
from tqdm import tqdm
import timeit ## can remove after assessing speed of different methods

version no.: 0.4.0


In [2]:
def euclidean_distance(target_track, other_track, frame):
    try:
        idx0 = target_track.t.index(find_apoptosis_time(target_track, index = False)) ## could also do just ## apop_index
        idx1 = other_track.t.index(frame) ## t.index provides the index of that frame
    except:
        return np.inf
    
    dx = target_track.x[idx0] - other_track.x[idx1]
    dy = target_track.y[idx0] - other_track.y[idx1]
    
    return np.sqrt(dx**2 + dy**2)

def euc_dist(target_track, other_track, frame, apop_index):
    try:
        idx0 = apop_index ##target_track.t.index(apop_time) ## could also do just ## apop_index
        idx1 = other_track.t.index(frame) ## t.index provides the index of that frame
    except:
        return np.inf
    
    dx = target_track.x[idx0] - other_track.x[idx1]
    dy = target_track.y[idx0] - other_track.y[idx1]
    
    return np.sqrt(dx**2 + dy**2)

def euc_dist_mito(target_track, other_track, apop_index):
    idx0 = apop_index ##target_track.t.index(apop_time) ## could also do just ## apop_index
    idx1 = -1 ## t[-1] is the final frame, ie the frame of mitosis
    
    dx = target_track.x[idx0] - other_track.x[idx1]
    dy = target_track.y[idx0] - other_track.y[idx1]
    
    return np.sqrt(dx**2 + dy**2)

def find_apoptosis_time(target_track, index): ### if index is set to True then the index of the apoptotic time (wrt target_track) is returned
    try:
        if target_track.label[0] == 'APOPTOSIS': ## if the first classification is apoptosis then thats a duff track
            print("False apoptosis Scr ID:", target_track.ID)
            apop_index = apop_time = False
            if index == True: 
                return apop_index
            else: 
                return apop_time
        else: 
            for i, j in enumerate(target_track.label):
                if j == 'APOPTOSIS' and target_track.label[i+1] == 'APOPTOSIS' and target_track.label[i+2] == 'APOPTOSIS': # and target_track.label[i+3] =='APOPTOSIS' and target_track.label[i+4] =='APOPTOSIS':
                    apop_index = i
                    break
            apop_time = target_track.t[apop_index]
            if index == True: 
                return apop_index
            else: 
                return apop_time
    except: 
        print("False apoptosis Scr ID:", target_track.ID)
        apop_index = apop_time = False
        if index == True: 
            return apop_index
        else: 
            return apop_time
    
def kymo_labels(num_bins, label_freq):
    radial_bin = radius / num_bins
    temporal_bin = t_range*2 / num_bins
#     print("Radius, time_range:", radius,",", (t_range*2))
#     print("Size radial bin:", radial_bin)
#     print("Size temporal bin:", temporal_bin)
#     print("No. bins:",num_bins)

    ## label frequence, ie num labels per bin
    #label_freq = 1

    ### generate labels for axis PIXELS/FRAMES
    xlocs = range(0, num_bins,label_freq) ## step of 2 to avoid crowding
    xlabels = []
    for m in range(int(-num_bins/2), int(num_bins/2),label_freq):
        xlabels.append(str(int((temporal_bin)*m)) + "," + str(int((temporal_bin)*m+temporal_bin)))
    #print("xlabels:",xlabels, type(xlabels))

    ylocs = range(0, num_bins, label_freq) ## step of 2 to avoid crowding
    ylabels = []
    for m in range(num_bins, 0, -label_freq):
        ylabels.append(str(int((radial_bin)*m)) + "," + str(int((radial_bin)*(m-1))))
    #print("ylabels:", ylabels, type(ylabels))
    
    return xlocs, xlabels, ylocs, ylabels

In [3]:
expt_path = '/home/nathan/data/kraken/h2b/giulia/GV0807/'
tracks_fn = os.path.join(expt_path, 'Pos3/Pos3_aligned/HDF/segmented.hdf5')
with btrack.dataio.HDF5FileHandler(tracks_fn, 'r', obj_type = "obj_type_1") as h:
    wt_tracks = h.tracks
with btrack.dataio.HDF5FileHandler(tracks_fn, 'r', obj_type = "obj_type_2") as h:
    scr_tracks = h.tracks
for i in range(len(scr_tracks)):
    scr_tracks[i].ID = -(scr_tracks[i].ID)
all_tracks = wt_tracks + scr_tracks ### negative IDs are scribble cells
print("Track information loaded and ordered according to cell type")

[INFO][2021/01/22 12:50:10 AM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5...
[INFO][2021/01/22 12:50:10 AM] Loading tracks/obj_type_1
[INFO][2021/01/22 12:50:13 AM] Loading objects/obj_type_1 (408973, 5) (388394 filtered: area>=100)
[INFO][2021/01/22 12:50:16 AM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5
[INFO][2021/01/22 12:50:16 AM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5...
[INFO][2021/01/22 12:50:16 AM] Loading tracks/obj_type_2
[INFO][2021/01/22 12:50:16 AM] Loading objects/obj_type_2 (12115, 5) (8894 filtered: area>=100)
[INFO][2021/01/22 12:50:16 AM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5


Track information loaded and ordered according to cell type


# to-do week from 21st Jan:

- [x] check the order of the histogram (col, row) in right orientation
- [x] put scr_tracks.IDs into the IMAGINARY (-ve)
- [x] combine wt_tracks + scr_tracks, retain option to just inc one (see above)
- [x] save out histograms as .npy arrays
- [ ] get list of real apoptoses from Chris
- [ ] sanity check for doubling times, shd be on the order of doubling cell number
- [x] input correct bin edges


# Calculate raw data for one cell (calculate neighbouring no. WT and WT mitoses)

In [4]:
print("Input cell type: 'WT' or 'Scr'")
cell_type = input()
print("Input cell ID")
cell_ID = int(input())

if cell_type == 'Scr':
    #index = [i for i, scr_apop in enumerate(scr_tracks) if scr_apop.ID == cell_ID][0]
    target_track = [scr_track for scr_track in scr_tracks if scr_track.ID == cell_ID][0]
    
    #target_track = scr_tracks[index]##### Show the first classification of each track
    ### if a scr cell is picked, the focal timepoint is its apoptosis
    apop_time = find_apoptosis_time(target_track, index = False)
    apop_index = find_apoptosis_time(target_track, index = True)
    
if cell_type == 'WT':
    index = [i for i, wt_track in enumerate(wt_tracks) if wt_track.ID == cell_ID][0]
    target_track = wt_tracks[index]##### Show the first classification of each track
    ### if a wt cell is picked, the focal time point is a random point in its track as this measurement will serve as a control
    apop_time = random.choice(target_track.t)
    apop_index = target_track.t.index(apop_time)

Input cell type: 'WT' or 'Scr'
WT
Input cell ID
300


In [5]:
target_track

,ID,t,x,y,z,parent,root,state,generation,dummy
0,300,414,54.993549,577.580627,0.0,125,34,3,2,False
1,300,415,56.124001,594.559998,0.0,125,34,3,2,False
2,300,416,52.631401,592.054626,0.0,125,34,3,2,False
3,300,417,49.265980,586.400024,0.0,125,34,3,2,False
4,300,418,50.284733,580.641479,0.0,125,34,0,2,False
5,300,419,51.322884,577.893433,0.0,125,34,0,2,False
6,300,420,52.107830,575.512573,0.0,125,34,0,2,False
7,300,421,49.215084,572.993042,0.0,125,34,0,2,False
8,300,422,47.953270,570.281738,0.0,125,34,0,2,False
9,300,423,45.363270,566.977234,0.0,125,34,0,2,False


#### Pick a temporal and radial range to scan over

In [6]:
radius = 400
t_range = 200 ### actual range will be twice this as it scans neg and pos

#### Scanning over just wts

In [ ]:
start = timeit.default_timer()
wts = [tuple(((wt_track.ID),
                  (round((euc_dist(target_track, wt_track, (apop_time+delta_t), apop_index)),2)),
                  ((apop_time + delta_t)))) 
           for delta_t in range(-t_range, t_range)
           for wt_track in wt_tracks 
               if euc_dist(target_track, wt_track, apop_time + delta_t, apop_index)<radius
           ]
stop = timeit.default_timer()

print('WT cell counter calculation time: ', round((stop - start),2), 'seconds') 

In [ ]:
wts

In [ ]:
start = timeit.default_timer()
### count number of mitoses
wts_mito = [tuple(((wt_track.ID),
                  (round((euc_dist_mito(target_track, wt_track, apop_index)),2)),
                  (wt_track.t[-1]),
                  (wt_track.label[-1]),
                  (wt_track.fate.name))) 
           for wt_track in wt_tracks
            if 
            euc_dist_mito(target_track, wt_track, apop_index)<radius and ## within the full radial extent
            wt_track.fate.name == "DIVIDE" 
           ]
stop = timeit.default_timer()

print('WT mitosis cell counter calculation time: ', round((stop - start),2), 'seconds') 

In [ ]:
wts_mito

In [ ]:
#os.system('spd-say "calculation complete, objective achieved,faster, better, harder,stronger"')

# Save measurments out

In [9]:
if cell_type == 'Scr':
    raw_numbers_path = os.path.join(expt_path, 'analysis/raw_numbers')
    no_wt_fn = os.path.join(raw_numbers_path,('no_wt_Scr_ID_' + str(cell_ID) + '.csv'))
    no_wt_mito_fn = os.path.join(raw_numbers_path, ('no_wt_mito_Scr_ID_' + str(cell_ID) + '.csv'))
if cell_type == 'WT':
    raw_numbers_path = os.path.join(expt_path, 'analysis/raw_numbers/controls')
    no_wt_fn = os.path.join(raw_numbers_path,('no_wt_WT_ID_' + str(cell_ID) + '.csv'))
    no_wt_mito_fn = os.path.join(raw_numbers_path, ('no_wt_mito_WT_ID_' + str(cell_ID) + '.csv'))

In [ ]:
with open(no_wt_fn, 'w') as filehandle:
    filehandle.write('%s\n'%'ID, Distance, Frame')
    for listitem in wts:
        filehandle.write('%s\n' % (re.sub(r"[\(\)]", "",str(listitem))))

In [ ]:
with open(no_wt_mito_fn, 'w') as filehandle:
    filehandle.write('%s\n'%'ID, Distance, Frame, State, Fate')
    for listitem in wts_mito:
        filehandle.write('%s\n' % (re.sub(r"[\(\)]", "",str(listitem))))

# Plotting data from one event into kymographs

#### Getting xy variables

In [ ]:
distance = [wts[i][1] for i in range(0,len(wts))]
time = [wts[i][2] for i in range(0,len(wts))]

In [ ]:
distance_mito = [wts_mito[i][1] for i in range(0,len(wts_mito))]
time_mito = [wts_mito[i][2] for i in range(0,len(wts_mito))]

# Plotting number of wild_type only

In [ ]:
num_bins = 20
time_bin_edges = np.linspace((-(t_range)+apop_time),(t_range+apop_time), num_bins+1) ## 2dimensionalise
distance_bin_edges = np.linspace(0,radius, num_bins+1) ## 2dimensionalise
H_num_wt, x_autolabels, y_autolabels = np.histogram2d(distance,time, bins=[distance_bin_edges,time_bin_edges])
xlocs, xlabels, ylocs, ylabels = kymo_labels(num_bins, label_freq = int(num_bins*0.1)) ## 0.1 only adds ten labels to the kymograph
plt.xlabel("Number of frames since t = " + str(apop_time)+ ' (frames)')
plt.ylabel("Distance from apoptosis (pixels)")
if cell_type == 'WT':
    title = 'Kymograph centered around WT ID:' + str(target_track.ID) + ' at t=' +str(apop_time)##cell type and ID
    save_title = 'Kymo_no._wt_WT.png'
    no_wt_fn = os.path.join(expt_path, 'analysis/single_cell/controls',('no_wt_WT_ID_' + str(cell_ID) + '.png'))
if cell_type == 'Scr':
    title = 'Kymograph centered around Scr ID:' + str(target_track.ID) + ' at t=' +str(apop_time) 
    no_wt_fn = os.path.join(expt_path, 'analysis/single_cell',('no_wt_Scr_ID_' + str(cell_ID) + '.png'))
plt.title(title)
plt.imshow(np.flipud(H_num_wt))
plt.yticks(ylocs, ylabels)
plt.xticks(xlocs, xlabels, rotation='vertical')
plt.colorbar(label = 'No. of wild-type cell appearances')#, ticks = list(range(0, int(round(np.amax(H))),30)))
expt_label = 'expt:' + expt_path[36:42] + '\n 90:10 WT:Scr'
plt.text(num_bins+1,num_bins+3,expt_label)
plt.savefig(no_wt_fn, bbox_inches = 'tight', dpi = 300)
np.save(no_wt_fn[0:-4],H_num_wt)

In [ ]:
num_bins = 20
H_num_mito, x_autolabels, y_autolabels = np.histogram2d(distance_mito,time_mito, bins=[distance_bin_edges,time_bin_edges])
#H_mito = H_mito/bin_size ### because for every bin each cell could be counted for every frame present. Instead of reducing the count down to any instance of a cell, I want to preserve this cumulative count inside a single bin as a measure for how present that cell is in any bin. Ie a cell that is fully present will be counted fully, but only a fleeting presence of 1/20 frames will count as 1/20th of a fully presence. 
xlocs, xlabels, ylocs, ylabels = kymo_labels(num_bins, label_freq = int(num_bins*0.1)) ## 0.1 only adds ten labels to the kymograph
plt.xlabel("Number of frames since t = " + str(apop_time)+ ' (frames)')
plt.ylabel("Distance from apoptosis (pixels)")
if cell_type == 'WT':
    title = 'Kymograph centered around WT ID:' + str(target_track.ID) + ' at t=' +str(apop_time)##cell type and ID
    no_wt_mito_fn = os.path.join(expt_path, 'analysis/single_cell/controls', ('no_wt_mito_WT_ID_' + str(cell_ID) + '.png'))
if cell_type == 'Scr':
    title = 'Kymograph centered around Scr ID:' + str(target_track.ID) + ' at t=' +str(apop_time) 
    no_wt_mito_fn = os.path.join(expt_path, 'analysis/single_cell', ('no_wt_mito_Scr_ID_' + str(cell_ID) + '.png'))
plt.title(title)
plt.imshow(np.flipud(H_num_mito))
plt.yticks(ylocs, ylabels)
plt.xticks(xlocs, xlabels, rotation='vertical')
plt.colorbar(label = 'No. of wild-type cell mitoses')#, ticks = list(range(0, int(round(np.amax(H_mito))),2)))
expt_label = 'expt:' + expt_path[36:42] + '\n 90:10 WT:Scr'
plt.text(num_bins+1,num_bins+4,expt_label)
plt.savefig(no_wt_mito_fn, bbox_inches = 'tight', dpi = 300)
np.save(no_wt_mito_fn[0:-4],H_num_mito)

In [ ]:
num_bins = 20
H_prob_mito = H_num_mito/(H_num_wt+1E-10)
xlocs, xlabels, ylocs, ylabels = kymo_labels(num_bins, label_freq = int(num_bins*0.1)) ## 0.1 only adds ten labels to the kymograph
plt.xlabel("Number of frames since t = " + str(apop_time)+ ' (frames)')
plt.ylabel("Distance from apoptosis (pixels)")
if cell_type == 'WT':
    title = 'Kymograph centered around WT ID:' + str(target_track.ID) + ' at t=' +str(apop_time)##cell type and ID
    prob_wt_mito_fn = os.path.join(expt_path, 'analysis/single_cell/controls', ('prob_wt_mito_WT_ID_' + str(cell_ID) + '.png'))
if cell_type == 'Scr':
    title = 'Kymograph centered around Scr ID:' + str(target_track.ID) + ' at t=' +str(apop_time) 
    prob_wt_mito_fn = os.path.join(expt_path, 'analysis/single_cell', ('prob_wt_mito_Scr_ID_' + str(cell_ID) + '.png'))
plt.title(title)
plt.imshow(np.flipud(H_prob_mito))
plt.yticks(ylocs, ylabels)
plt.xticks(xlocs, xlabels, rotation='vertical')
plt.colorbar(label = 'Probability of wild-type cell mitoses')
expt_label = 'expt:' + expt_path[36:42] + '\n 90:10 WT:Scr'
plt.text(num_bins, num_bins+2,expt_label)
plt.savefig(prob_wt_mito_fn, bbox_inches = 'tight', dpi = 300)
np.save(prob_wt_mito_fn[0:-4],H_prob_mito)

# Merging all above into single block (calc for WT cells only)

In [ ]:
print("Input cell type: 'WT' or 'Scr'")
cell_type = input()
print("Input cell ID")
cell_ID = int(input())

if cell_type == 'Scr':
    #index = [i for i, scr_apop in enumerate(scr_tracks) if scr_apop.ID == cell_ID][0]
    target_track = [scr_track for scr_track in scr_tracks if scr_track.ID == cell_ID][0]
    
    #target_track = scr_tracks[index]##### Show the first classification of each track
    ### if a scr cell is picked, the focal timepoint is its apoptosis
    apop_time = find_apoptosis_time(target_track, index = False)
    apop_index = find_apoptosis_time(target_track, index = True)
    
if cell_type == 'WT':
    index = [i for i, wt_track in enumerate(wt_tracks) if wt_track.ID == cell_ID][0]
    target_track = wt_tracks[index]##### Show the first classification of each track
    ### if a wt cell is picked, the focal time point is a random point in its track as this measurement will serve as a control
    apop_time = random.choice(target_track.t)
    apop_index = target_track.t.index(apop_time)

radius = 400
t_range = 200 ### actual range will be twice this as it scans neg and pos

start = timeit.default_timer()
wts = [tuple(((wt_track.ID),
                  (round((euc_dist(target_track, wt_track, (apop_time+delta_t), apop_index)),2)),
                  ((apop_time + delta_t)))) 
           for delta_t in range(-t_range, t_range)
           for wt_track in wt_tracks 
               if euc_dist(target_track, wt_track, apop_time + delta_t, apop_index)<radius
           ]
stop = timeit.default_timer()

print('WT cell counter calculation time: ', round((stop - start),2), 'seconds') 

start = timeit.default_timer()
### count number of mitoses
wts_mito = [tuple(((wt_track.ID),
                  (round((euc_dist_mito(target_track, wt_track, apop_index)),2)),
                  (wt_track.t[-1]),
                  (wt_track.label[-1]),
                  (wt_track.fate.name))) 
           for wt_track in wt_tracks
            if 
            euc_dist_mito(target_track, wt_track, apop_index)<radius and ## within the full radial extent
            wt_track.fate.name == "DIVIDE" 
           ]
stop = timeit.default_timer()

print('WT mitosis cell counter calculation time: ', round((stop - start),2), 'seconds') 

#os.system('spd-say "calculation complete, objective achieved,faster, better, harder,stronger"')

if cell_type == 'Scr':
    raw_numbers_path = os.path.join(expt_path, 'analysis/raw_numbers')
    no_wt_fn = os.path.join(raw_numbers_path,('no_wt_Scr_ID_' + str(cell_ID) + '.csv'))
    no_wt_mito_fn = os.path.join(raw_numbers_path, ('no_wt_mito_Scr_ID_' + str(cell_ID) + '.csv'))
if cell_type == 'WT':
    raw_numbers_path = os.path.join(expt_path, 'analysis/raw_numbers/controls')
    no_wt_fn = os.path.join(raw_numbers_path,('no_wt_WT_ID_' + str(cell_ID) + '.csv'))
    no_wt_mito_fn = os.path.join(raw_numbers_path, ('no_wt_mito_WT_ID_' + str(cell_ID) + '.csv'))

with open(no_wt_fn, 'w') as filehandle:
    filehandle.write('%s\n'%'ID, Distance, Frame')
    for listitem in wts:
        filehandle.write('%s\n' % (re.sub(r"[\(\)]", "",str(listitem))))

with open(no_wt_mito_fn, 'w') as filehandle:
    filehandle.write('%s\n'%'ID, Distance, Frame, State, Fate')
    for listitem in wts_mito:
        filehandle.write('%s\n' % (re.sub(r"[\(\)]", "",str(listitem))))

distance = [wts[i][1] for i in range(0,len(wts))]
time = [wts[i][2] for i in range(0,len(wts))]

distance_mito = [wts_mito[i][1] for i in range(0,len(wts_mito))]
time_mito = [wts_mito[i][2] for i in range(0,len(wts_mito))]

## choose bin size 
num_bins = 20

time_bin_edges = np.linspace((-(t_range)+apop_time),(t_range+apop_time), num_bins+1) ## 2dimensionalise
distance_bin_edges = np.linspace(0,radius, num_bins+1) ## 2dimensionalise
H_num_wt, x_autolabels, y_autolabels = np.histogram2d(distance,time, bins=[distance_bin_edges,time_bin_edges])
xlocs, xlabels, ylocs, ylabels = kymo_labels(num_bins, label_freq = int(num_bins*0.1)) ## 0.1 only adds ten labels to the kymograph
plt.xlabel("Number of frames since t = " + str(apop_time)+ ' (frames)')
plt.ylabel("Distance from apoptosis (pixels)")
if cell_type == 'WT':
    title = 'Kymograph centered around WT ID:' + str(target_track.ID) + ' at t=' +str(apop_time)##cell type and ID
    save_title = 'Kymo_no._wt_WT.png'
    no_wt_fn = os.path.join(expt_path, 'analysis/single_cell/controls',('no_wt_WT_ID_' + str(cell_ID) + '.png'))
if cell_type == 'Scr':
    title = 'Kymograph centered around Scr ID:' + str(target_track.ID) + ' at t=' +str(apop_time) 
    no_wt_fn = os.path.join(expt_path, 'analysis/single_cell',('no_wt_Scr_ID_' + str(cell_ID) + '.png'))
plt.title(title)
plt.imshow(np.flipud(H_num_wt))
plt.yticks(ylocs, ylabels)
plt.xticks(xlocs, xlabels, rotation='vertical')
plt.colorbar(label = 'No. of wild-type cell appearances')#, ticks = list(range(0, int(round(np.amax(H))),30)))
expt_label = 'expt:' + expt_path[36:42] + '\n 90:10 WT:Scr'
plt.text(num_bins+1,num_bins+3,expt_label)
plt.savefig(no_wt_fn, bbox_inches = 'tight', dpi = 300)
np.save(no_wt_fn[0:-4],H_num_wt)
plt.clf()
num_bins = 20
H_num_mito, x_autolabels, y_autolabels = np.histogram2d(distance_mito,time_mito, bins=[distance_bin_edges,time_bin_edges])
#H_mito = H_mito/bin_size ### because for every bin each cell could be counted for every frame present. Instead of reducing the count down to any instance of a cell, I want to preserve this cumulative count inside a single bin as a measure for how present that cell is in any bin. Ie a cell that is fully present will be counted fully, but only a fleeting presence of 1/20 frames will count as 1/20th of a fully presence. 
xlocs, xlabels, ylocs, ylabels = kymo_labels(num_bins, label_freq = int(num_bins*0.1)) ## 0.1 only adds ten labels to the kymograph
plt.xlabel("Number of frames since t = " + str(apop_time)+ ' (frames)')
plt.ylabel("Distance from apoptosis (pixels)")
if cell_type == 'WT':
    title = 'Kymograph centered around WT ID:' + str(target_track.ID) + ' at t=' +str(apop_time)##cell type and ID
    no_wt_mito_fn = os.path.join(expt_path, 'analysis/single_cell/controls', ('no_wt_mito_WT_ID_' + str(cell_ID) + '.png'))
if cell_type == 'Scr':
    title = 'Kymograph centered around Scr ID:' + str(target_track.ID) + ' at t=' +str(apop_time) 
    no_wt_mito_fn = os.path.join(expt_path, 'analysis/single_cell', ('no_wt_mito_Scr_ID_' + str(cell_ID) + '.png'))
plt.title(title)
plt.imshow(np.flipud(H_num_mito))
plt.yticks(ylocs, ylabels)
plt.xticks(xlocs, xlabels, rotation='vertical')
plt.colorbar(label = 'No. of wild-type cell mitoses')#, ticks = list(range(0, int(round(np.amax(H_mito))),2)))
expt_label = 'expt:' + expt_path[36:42] + '\n 90:10 WT:Scr'
plt.text(num_bins+1,num_bins+4,expt_label)
plt.savefig(no_wt_mito_fn, bbox_inches = 'tight', dpi = 300)
np.save(no_wt_mito_fn[0:-4],H_num_mito)
plt.clf()
num_bins = 20
H_prob_mito = H_num_mito/(H_num_wt+1E-10)
xlocs, xlabels, ylocs, ylabels = kymo_labels(num_bins, label_freq = int(num_bins*0.1)) ## 0.1 only adds ten labels to the kymograph
plt.xlabel("Number of frames since t = " + str(apop_time)+ ' (frames)')
plt.ylabel("Distance from apoptosis (pixels)")
if cell_type == 'WT':
    title = 'Kymograph centered around WT ID:' + str(target_track.ID) + ' at t=' +str(apop_time)##cell type and ID
    prob_wt_mito_fn = os.path.join(expt_path, 'analysis/single_cell/controls', ('prob_wt_mito_WT_ID_' + str(cell_ID) + '.png'))
if cell_type == 'Scr':
    title = 'Kymograph centered around Scr ID:' + str(target_track.ID) + ' at t=' +str(apop_time) 
    prob_wt_mito_fn = os.path.join(expt_path, 'analysis/single_cell', ('prob_wt_mito_Scr_ID_' + str(cell_ID) + '.png'))
plt.title(title)
plt.imshow(np.flipud(H_prob_mito))
plt.yticks(ylocs, ylabels)
plt.xticks(xlocs, xlabels, rotation='vertical')
plt.colorbar(label = 'Probability of wild-type cell mitoses')
expt_label = 'expt:' + expt_path[36:42] + '\n 90:10 WT:Scr'
plt.text(num_bins, num_bins+2,expt_label)
plt.savefig(prob_wt_mito_fn, bbox_inches = 'tight', dpi = 300)
np.save(prob_wt_mito_fn[0:-4],H_prob_mito)